In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model
import scipy.special as sp

from Source import AWGN

Using TensorFlow backend.


In [2]:
n_power=7
CodeRate=0.5
N_length=int(2**n_power)
msg_length=int(CodeRate*N_length)
frozen_length=N_length-msg_length
codeNum =10**2
test_codeNum = 10**3

EbNo_Start=1.0  # in dB
EbNo_delta=0.5 # in dB
EbNo_Numbers=10 
EbNo=np.arange(EbNo_Start,EbNo_Start+EbNo_Numbers*EbNo_delta,EbNo_delta)
ebno=10.**(EbNo/10.)          # dB to real value 변환
BER_BPSK_Uncoded=0.5*sp.erfc(np.sqrt(ebno))      # BPSK BER 공식

In [3]:
G_All=AWGN.AllGeneratorMatrixPolarCode(n_power) ###생성행렬생성
(Z_AWGN,Z_decending_index_AWGN)=AWGN.BhanttacharyyaParameterAWGN(N_length, n_power, CodeRate)
SelectedBitsPositions = Z_decending_index_AWGN[frozen_length:]
SelectedBitsPositions.sort()
FrozenBitsPositions = Z_decending_index_AWGN[:frozen_length]
FrozenBitsPositions.sort()

In [4]:
model = load_model('Model/model_2btis_AWGN_p6.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



In [5]:
file_buffer = 'Result/AWGN_model_p6.csv'
model_f = open(file_buffer, 'w')
file_buffer = 'Result/AWGN.csv'
AWGN_f = open(file_buffer, 'w')
file_buffer = 'Result/AWGN_only_model.csv'
AWGN_only_model_f = open(file_buffer, 'w')

In [7]:
AWGN_BERs = []
Model_BERs = []
Only_BERs = []
for EbNo_cnt, EbNo_real in enumerate(ebno):
    print(EbNo_cnt)
    total_AWGN_BER = 0
    total_Moel_BER = 0
    total_only_BER = 0
    for i in range(10):
        msg, codeword, uncodedbits = AWGN.Encoding(codeNum, N_length, n_power, msg_length, SelectedBitsPositions, G_All[n_power]) 
        ReceivedAWGN = AWGN.ReceivedCode(EbNo_real, CodeRate,codeNum, N_length, codeword)
        model_predict_AWGN = model.predict(ReceivedAWGN, verbose = 0)
    
        LLR_AWGN = AWGN.LLR_Value_AWGN(EbNo_real, CodeRate, ReceivedAWGN)
        LLR_Model_AWGN = AWGN.LLR_Value_AWGN(EbNo_real, CodeRate, model_predict_AWGN)
        
        Esti_AWGN = AWGN.Decoding(codeNum, N_length, n_power, SelectedBitsPositions, 
                                FrozenBitsPositions, G_All, LLR_AWGN)
        Esti_Model_AWGN = AWGN.Decoding(codeNum, N_length, n_power, SelectedBitsPositions, 
                                FrozenBitsPositions, G_All, LLR_Model_AWGN)
        
        AWGN_BER = AWGN.BERCalculation(Esti_AWGN, msg, codeNum, msg_length)
        Model_BER = AWGN.BERCalculation(Esti_Model_AWGN, msg, codeNum, msg_length)
        only_BER = AWGN.BERCalculation(np.where(model_predict_AWGN < 0, 1 ,0)[:,SelectedBitsPositions], msg, codeNum, msg_length)
        
        total_AWGN_BER += AWGN_BER
        total_Moel_BER += Model_BER
        total_only_BER += only_BER
        
    AWGN_BERs.append(total_AWGN_BER/10)
    Model_BERs.append(total_Moel_BER/10)
    Only_BERs.append(total_only_BER/10)
    
    write_buffer = 'EbNo, %f,' %(EbNo[EbNo_cnt])
    model_f.write(write_buffer)
    AWGN_f.write(write_buffer)
    AWGN_only_model_f.write(write_buffer)
    
    write_buffer = 'BER, %f\n' %(AWGN_BERs[EbNo_cnt])
    AWGN_f.write(write_buffer)
    write_buffer = 'BER, %f\n' %(Model_BERs[EbNo_cnt])
    model_f.write(write_buffer)
    write_buffer = 'BER, %f\n' %(Only_BERs[EbNo_cnt])
    AWGN_only_model_f.write(write_buffer)
    
model_f.close()
AWGN_f.close()
AWGN_only_model_f.close()

0
1
2
3
4
5
6
7
8
9
